In [24]:
import pandas as pd
import numpy as np
import re
import string
import math
import hashlib
import os

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# EDA

### Data Loading

In [25]:
with open(os.path.join("data", "train.csv")) as f:
    # read the training dataset
    df = pd.read_csv(f, index_col = 'id')

with open(os.path.join("data", "test.csv")) as f:
    # read the test dataset
    X_test = pd.read_csv(f, index_col  ='id')



X = df.drop(['Hardness'], axis=1)
y = df['Hardness']

In [27]:
df.head()

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.860,0.496070,0.91457,6.0
1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.910,0.492719,0.71760,6.5
2,76.0,8.885992,15.6,5.6,33.739258,12.08630,2.828,1.788,0.864,0.481478,1.50633,2.5
3,100.0,8.795296,10.0,4.8,20.213349,10.94850,2.648,1.626,0.936,0.489272,0.78937,6.0
4,116.0,9.577996,11.6,4.8,24.988133,11.82448,2.766,1.682,0.896,0.492736,1.86481,6.0


In [28]:
X_test.head()

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
id,,,,,,,,,,,
10407,884.0,121.420000,35.36,5.28,82.561240,9.370384,2.298,1.9072,1.200000,0.461899,1.79459
10408,90.0,9.931960,18.00,5.60,39.568056,12.086300,2.868,1.6520,0.864000,0.476196,1.41194
10409,116.0,7.767992,11.60,4.80,23.231818,11.023840,2.644,1.7940,0.960000,0.499514,0.78834
10410,100.0,9.107996,10.00,4.80,20.298893,12.086300,2.828,1.6620,0.792000,0.495796,1.20466
10411,55.0,4.030000,11.00,4.00,22.977675,11.280950,2.455,1.7500,0.893333,0.476095,0.93456


### Data Description

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10407 entries, 0 to 10406
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   allelectrons_Total     10407 non-null  float64
 1   density_Total          10407 non-null  float64
 2   allelectrons_Average   10407 non-null  float64
 3   val_e_Average          10407 non-null  float64
 4   atomicweight_Average   10407 non-null  float64
 5   ionenergy_Average      10407 non-null  float64
 6   el_neg_chi_Average     10407 non-null  float64
 7   R_vdw_element_Average  10407 non-null  float64
 8   R_cov_element_Average  10407 non-null  float64
 9   zaratio_Average        10407 non-null  float64
 10  density_Average        10407 non-null  float64
 11  Hardness               10407 non-null  float64
dtypes: float64(12)
memory usage: 1.0 MB


In [30]:
df.columns

Index(['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average', 'Hardness'],
      dtype='object')

In [31]:
df.head()

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.860,0.496070,0.91457,6.0
1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.910,0.492719,0.71760,6.5
2,76.0,8.885992,15.6,5.6,33.739258,12.08630,2.828,1.788,0.864,0.481478,1.50633,2.5
3,100.0,8.795296,10.0,4.8,20.213349,10.94850,2.648,1.626,0.936,0.489272,0.78937,6.0
4,116.0,9.577996,11.6,4.8,24.988133,11.82448,2.766,1.682,0.896,0.492736,1.86481,6.0


In [32]:
df.describe()

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
count,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000,10407.000000
mean,128.053516,14.491342,17.033222,4.546789,37.507703,10.938308,2.607662,1.731330,0.944132,0.493349,2.132984,4.647126
std,224.123776,15.972877,10.468734,0.690864,26.012313,1.408276,0.334906,0.192481,0.180017,0.063080,1.936656,1.680525
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,68.000000,7.558488,10.000000,4.000000,20.298893,10.590660,2.530000,1.672500,0.864000,0.476196,0.814800,3.000000
50%,100.000000,10.650000,12.600000,4.714286,26.203827,11.202760,2.706000,1.732727,0.915556,0.488550,1.351550,5.500000
75%,131.000000,16.676996,22.000000,4.800000,48.719500,11.670725,2.805000,1.800000,0.981667,0.496070,2.741550,6.000000
max,15300.000000,643.093804,67.000000,6.000000,167.400000,15.245810,3.443000,2.250000,1.615840,0.825990,10.970000,10.000000


In [51]:
(df == 0).sum()

allelectrons_Total       76
density_Total            54
allelectrons_Average     76
val_e_Average            71
atomicweight_Average     77
ionenergy_Average        73
el_neg_chi_Average       67
R_vdw_element_Average    68
R_cov_element_Average    77
zaratio_Average          62
density_Average          56
Hardness                  0
exp_feature              74
dtype: int64

### Data Analysis

In [33]:
fig = px.histogram(df, x = 'Hardness')
fig.update_layout(yaxis_title='Count', width = 800, height = 400)
fig.show()

The dataset is unbalanced, with a lower representation of harder materials.

In [34]:
features = df.columns

# for feat_i in features:
#         fig = px.scatter(df, x = 'Hardness', y = feat_i) #, color_discrete_sequence=['#2bb5b3'])
#         fig.update_layout(yaxis_title = feat_i, xaxis_title = "Hardness", width = 750, height = 400)
#         fig.show()
#         str_i = "images/hardness_vs_"+str(feat_i)+".png"
#         fig.write_image(str_i)

In [35]:
corr_mat = df.corr(method = 'pearson')
 
lower_triangle_mask = np.triu(np.ones(corr_mat.shape)).astype(np.bool)
corr_mat_lower = corr_mat.mask(lower_triangle_mask)

fig = px.imshow(corr_mat_lower, text_auto=True, aspect="auto")
fig.show()

/var/folders/vr/h0tx7q8j69zgbwswj_d_cxgm0000gp/T/ipykernel_10206/1078476561.py:3: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [36]:
identity_matrix = np.identity(corr_mat.shape[0]).astype(bool)  # diagonal only 
corr_mat_without_diagonal = corr_mat.mask(identity_matrix)
corr_mat_without_diagonal.loc['Hardness'].map(lambda x: abs(x)).sort_values(ascending=False)

atomicweight_Average     0.402948
allelectrons_Average     0.400496
density_Average          0.360780
el_neg_chi_Average       0.290352
ionenergy_Average        0.203203
R_cov_element_Average    0.189111
val_e_Average            0.161715
density_Total            0.161524
allelectrons_Total       0.061687
R_vdw_element_Average    0.061402
zaratio_Average          0.055248
Hardness                      NaN
Name: Hardness, dtype: float64

Hardness is more correlated with:
> 1. average atomic weight  
> 2. average number of electrons
> 3. average density   
> 4. average Pauling electronegativity
> 5. average ionic energy
> 6. covalent atomic radii
> 7. number of valence electrons

In [37]:
corr_mat = df.corr(method = 'spearman')
 
lower_triangle_mask = np.triu(np.ones(corr_mat.shape)).astype(np.bool)
corr_mat_lower = corr_mat.mask(lower_triangle_mask)

identity_matrix = np.identity(corr_mat.shape[0]).astype(bool)  # diagonal only 
corr_mat_without_diagonal = corr_mat.mask(identity_matrix)
corr_mat_without_diagonal.loc['Hardness'].map(lambda x: abs(x)).sort_values(ascending=False)

/var/folders/vr/h0tx7q8j69zgbwswj_d_cxgm0000gp/T/ipykernel_10206/2947429821.py:3: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



zaratio_Average          0.288997
allelectrons_Average     0.277176
atomicweight_Average     0.274760
el_neg_chi_Average       0.263644
density_Average          0.235580
R_cov_element_Average    0.214552
R_vdw_element_Average    0.181879
density_Total            0.155561
val_e_Average            0.129981
ionenergy_Average        0.111221
allelectrons_Total       0.044612
Hardness                      NaN
Name: Hardness, dtype: float64

In [38]:
corr_mat_lower_unstacked = corr_mat_lower.unstack().dropna()
two_feat_corr = corr_mat_lower_unstacked
two_feat_corr.nlargest(5)

allelectrons_Average   atomicweight_Average     0.991679
atomicweight_Average   density_Average          0.834386
allelectrons_Average   density_Average          0.832844
R_vdw_element_Average  R_cov_element_Average    0.799929
allelectrons_Average   R_cov_element_Average    0.716820
dtype: float64

Plotting top 5 two-feature correlations.

In [39]:
for i in two_feat_corr.nlargest(5).index:
    feat_tuple = i
    x_i = i[0]
    y_i = i[1]
    fig = px.scatter(df, x = x_i, y = y_i, color = 'Hardness') #, color_discrete_sequence=['#2bb5b3'])
    fig.update_layout(yaxis_title = y_i, xaxis_title = x_i, width = 750, height = 400)
    fig.show()
    # str_i = "images/hardness_vs_"+str(feat_i)+".png"
    # fig.write_image(str_i)


From the plots above, allelectrons_Average and atomicweight_Average seem to be linearly correlated. As such, only one of them will be used for modelling: allelectrons_Average, as it is the most correlated with Hardness.

## Labelling hardness values according to their quartile

Different hardness values and ranges might present different relations with the dataset features.

In [41]:
df.Hardness.describe()

count    10407.000000
mean         4.647126
std          1.680525
min          1.000000
25%          3.000000
50%          5.500000
75%          6.000000
max         10.000000
Name: Hardness, dtype: float64

In [48]:
def hardness_quartile(x):
    q1 = 3.0
    q2 = 5.5
    q3 = 6.0

    if 0 < x <= q1:
        return 'q1'
    elif q1 < x <= q2:
        return 'q2'
    elif q2 < x <= q3:
        return 'q3'
    elif q3 < x <= 10:
        return 'q4'

df_new_feats = df.copy()
df_new_feats['quartile'] = df_new_feats['Hardness'].apply(hardness_quartile)

In [47]:
fig = px.scatter(df_new_feats, x = 'allelectrons_Average', y='atomicweight_Average', color = 'quartile')
fig.update_layout(yaxis_title = 'atomicweight_Average', xaxis_title = 'allelectrons_Average', width = 750, height = 400)
fig.show()

In [49]:
df_new_feats.quartile.value_counts()

q2    3797
q1    2826
q3    2476
q4    1308
Name: quartile, dtype: int64